# Aimsun Macrosimulation SQLite Output Database Analyzer

### Goal of the notebook

This purpose of this Jupyter notebook is to process the database produced by the Aimsun static macrosimulations.
***
**Outputs:** 

Comparsion biplots:
- compare_link_flow.png


**Inputs:** 

Aimsun macrosimulation outputs:
- aimsun-outputs.sqlite
    - The database should at least contain the following tables: SIM_INFO, MASECT.

Ground truth data:
- flow_processed_2019.csv

**Dependent scripts:** 
None

 **IMPORTANT:** Run the iPython cell below in order to import the necessary packages.

In [1]:
# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from fremontdropbox import get_dropbox_location

path_dropbox = get_dropbox_location()
data_folder = os.path.join(path_dropbox, 'Private Structured data collection')
sql_folder = os.path.join(data_folder, 'Aimsun','Outputs')

In [15]:
import numpy as np
import scipy
import matplotlib
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

### SQLite Helper Functions

These functions are wrapper functions for the underlying SQLite commands we use to query data from the SQLite database. The notebook assumes that the Aimsun database file has been placed into the current directory of the notebook.

In [3]:
def create_connection(db_file):
    conn = sqlite3.connect(db_file)
    return conn

def create_df_from_sql_table(conn, table_name):
    query = conn.execute("SELECT * From {}".format(table_name))
    cols = [column[0] for column in query.description]
    results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
    return results

def select_all_from_table(conn, table, should_print = True):
    cur = conn.cursor()
    if should_print:
        # Prevents us from accidentally clogging up the notebook with huge print statements
        cur.execute("SELECT * FROM {} LIMIT 10".format(table))
    else:
        cur.execute("SELECT * FROM {}".format(table))
    rows = cur.fetchall()
    if should_print:
        for row in rows:
            print(row)
    return rows

### SimulatorInfo Class (SIM_INFO Table)

This class parses information from the ```SIM_INFO``` table in the Aimsun database. This table stores meta information about the simulation in general, including the owner of the file and the version number currently in use.

In [7]:
class SimulatorInfo:
    
    def __init__(self, values):
        self.data_id = values[0]
        self.data_id_name = values[1]
        self.effective_data_id = values[2]
        self.uses_external_id = True if values[4] else False
        self.scenario_date = values[5]
        self.start_time = values[6]
        self.duration = values[7]
        self.rand_seed = values[8]
        self.type = 'Simulated Data' if values[9] == 1 else 'Average'
        self.warm_up_time = values[10]
        self.sim_model = values[11]
        self.aimsun_version = values[12]
        self.num_iters = values[13]
        self.exec_date = values[14]
        self.experiment_id = values[15]
        self.experiment_name = values[16]
        self.scenario_id = values[17]
        self.scenario_name = values[18]
        self.author = values[28]
        self.num_interval = (self.duration-self.warm_up_time)//900 # one interval per 15 mins

    def __str__(self):
        delimiter = ","
        return "Data ID: {}{}".format(self.data_id, delimiter) +\
            "Data ID Name: {}{}".format(self.data_id_name, delimiter) +\
            "Start Time: {}{}".format(self.start_time, delimiter) +\
            "Duration: {}{}".format(self.duration, delimiter) +\
            "Num intervals: {}{}".format(self.num_interval, delimiter) +\
            "Type: {}{}".format(self.type, delimiter) +\
            "Simulation Model: {}{}".format(self.sim_model, delimiter) +\
            "Execution Date: {}{}".format(self.exec_date, delimiter) +\
            "Scenarion Name: {}{}".format(self.scenario_name, delimiter) +\
            "Owner: {}".format(self.author)
    
    def __repr__(self):
        return str(self)

### Main AimsunAnalyzer Class

To be implemented. The skeleton code has been given below.

Tables ```MISECT```, ```MIVEHTRAJECTORY``` and ```MISECTVEHTRAJECTORY``` are converted into dataframes and stored before further analysis.

In [16]:
class AimsunAnalyzer:
    
    def __init__(self, simulation_file, simulation_filetype, ground_truth_file = None, ground_truth_filetype = None):
        """
        Initializes the Aimsun analyzer.
        
        @param simulation_file:          The file path of the source file of Aimsun macro/microsimulation outputs.
        @param simulation_filetype:      The type of the src_simulation file (can be .csv or .sqlite).
        @param ground_truth_file:        The file path of the source file of Aimsun macro/microsimulation outputs.
        @param ground_truth_filetype:    The type of the src_simulation file (can be .csv or .sqlite).
        """
        self.database = simulation_file
        self.conn = create_connection(self.database)
        print("=====Connection Established.=====")
        
        self.model_info = SimulatorInfo(select_all_from_table(self.conn, "SIM_INFO", should_print = False)[0])
        print("=====Model Information Loaded.=====")
        
        self.sections = create_df_from_sql_table(self.conn, "MASECT")

        print("=====Simulation Data Loaded.=====")
        
        self.ground_truth_file = ground_truth_file
        self.ground_truth_filetype = ground_truth_filetype
    
    def get_link_flow(self, road_id):
        """
        Returns the link flow (veh/h) for a road_id.
        @param road_id:         The external ID of the road in Aimsun in string format, i.e. "1242".
        @return:                The link flow on the road with road_id.
        """
        flow = self.sections[self.sections["eid"]==road_id]["flow"].sum()
        return flow
    
    def compare_flows(self, road_ids='All'):
        # may not be useful
        """
        Returns a matplotlib plot with comparison information on the flows for the given road IDs at the
        specified time intervals.

        @param road_ids:              A list of the road IDs to be compared.
        @param time_interval:         A list of the corresponding time intervals for the road IDs.

        @return:                      A comparison plot in Matplotlib for the link flows.
        @return:                      The list of link flows within the time interval between start_time
                                      and end_time on roads with road_ids.
        """
        if self.ground_truth_data is None:
            print("Error: No ground truth has been passed in.")


## Main Code

This section contains the code that should be run in order to generate results in this notebook. Below is the necessary starter code that connects to the database and outputs the current simulator info:

In [14]:
static_db = os.path.join(sql_folder, "2019_latest_0421_static.sqlite")
static_analyzer = AimsunAnalyzer(static_db, "sqlite")

od_adjustment_db = os.path.join(sql_folder, "2019_latest_0421_od_adjustment.sqlite")
od_adjustment_analyzer = AimsunAnalyzer(od_adjustment_db, "sqlite")

=====Connection Established.=====
=====Model Information Loaded.=====
=====Simulation Data Loaded.=====
=====Connection Established.=====
=====Model Information Loaded.=====
=====Simulation Data Loaded.=====


In [12]:
static_analyzer.get_link_flow("35500")

1377.1697917864321

In [17]:
od_adjustment_analyzer.get_link_flow("35500")

1863.2793944433279